### Codigo para extraer solo grabaciones nocturnas y moverlas a un directorio nuevo. 

In [ ]:
import os
import shutil
from pathlib import Path

# Function to check if a filename corresponds to night-time

def is_night_time(filename):
    time_str = filename.split('_')[1].split('.')[0]  # Extracts the time part from the filename
    return ('000000' <= time_str <= '070000') or ('180000' <= time_str <= '235000')

# The base folder containing the ARU analysis data
base_folder = '/mnt/e/DOCTORADO/grabaciones_disco1/day_recordings'  # Use raw string for Windows path
night_folder = '/mnt/e/DOCTORADO/grabaciones_disco1/night_recordings'  # Use raw string for Windows path

# Debugging
print(f"Starting at base folder: {base_folder}")

# Iterate through the folder hierarchy
for root, dirs, files in os.walk(base_folder):
    print(f"Root: {root}, Dirs: {dirs}, Files: {files[:3]}")  # Print the first 3 files for debugging
    for file in files:
        if file.endswith('.WAV'):
            #print(f"Checking file {file}...")  # Debug message
            if is_night_time(file):
                #print(f"Moving file {file}...")  # Debug message

                # Get the full path of the current file
                full_file_path = os.path.abspath(os.path.join(root, file))
                
                # Create the same folder structure in the night folder
                relative_path = os.path.relpath(root, base_folder)
                new_folder_path = os.path.abspath(os.path.join(night_folder, relative_path))
                Path(new_folder_path).mkdir(parents=True, exist_ok=True)

                # Move the file
                new_file_path = os.path.abspath(os.path.join(new_folder_path, file))
                shutil.move(full_file_path, new_file_path)
                
print("Successfully moved all night recordings.")


### Afinar la clasificacion de los audios basado en sunset y sunrise de cada fecha

In [ ]:
import os
import shutil
import pandas as pd
from datetime import datetime

# Load the sunrise and sunset data
sunrise_sunset_data = pd.read_csv('/mnt/d/Prueba/SunsetSunrise.csv')
# Convert the date column to the correct format to match the WAV filenames
sunrise_sunset_data['date'] = pd.to_datetime(sunrise_sunset_data['date'], format='%m/%d/%Y').dt.strftime('%Y%m%d')
# Set the 'date' column as the index for easier lookup
sunrise_sunset_data.set_index('date', inplace=True)
# Convert sunrise and sunset times to time objects
sunrise_sunset_data['sunrise'] = pd.to_datetime(sunrise_sunset_data['sunrise'], format='%H:%M:%S').dt.time
sunrise_sunset_data['sunset'] = pd.to_datetime(sunrise_sunset_data['sunset'], format='%H:%M:%S').dt.time

# Function to reorganize files based on sunrise and sunset times
def reorganize_files(source_folder, destination_folder, sunrise_sunset_data):
    for root, dirs, files in os.walk(source_folder):
        for file in files:
            if file.endswith('.WAV'):
                # Extract date and time from filename
                date_str, time_str = file.split('_')[0], file.split('_')[1].split('.')[0]
                file_date = date_str
                file_time = datetime.strptime(time_str, '%H%M%S').time()

                # Check if the date is in the sunrise/sunset data
                if file_date in sunrise_sunset_data.index:
                    sunrise_time = sunrise_sunset_data.loc[file_date, 'sunrise']
                    sunset_time = sunrise_sunset_data.loc[file_date, 'sunset']

                    # Determine if the file is a night or day recording
                    is_night_recording = not (sunrise_time <= file_time <= sunset_time)

                    # Construct the new directory path based on the current directory structure
                    relative_path = os.path.relpath(root, source_folder)
                    new_dir_path = os.path.join(destination_folder, relative_path)

                    # Move the file if it's in the incorrect folder
                    if ('night_recordings' in root and not is_night_recording) or ('day_recordings' in root and is_night_recording):
                        os.makedirs(new_dir_path, exist_ok=True)
                        shutil.move(os.path.join(root, file), os.path.join(new_dir_path, file))

# Paths to the folders
day_folder = '/mnt/d/Prueba/day_recordings/'
night_folder = '/mnt/d/Prueba/night_recordings/'

# Reorganize files in night recordings folder
reorganize_files(night_folder, day_folder, sunrise_sunset_data)

# Reorganize files in day recordings folder
reorganize_files(day_folder, night_folder, sunrise_sunset_data)

print("Files have been reorganized based on the sunrise and sunset times.")


### Codigo para reroganizar las gabaciones en ocasion de muestreo

In [5]:
import os
import shutil

# Define the base directory containing the date folders and SN folders
base_dir = '/mnt/d/night_recordings/CH12-ARU6-1907-0408/'  # Replace with your actual path

# Get a sorted list of all date folders
date_folders = sorted([d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d)) and d.isdigit()])

# Iterate over the date folders and pair them for each SN folder
for i in range(len(date_folders)-1):
    sn_folder = f"SN_{str(i+1).zfill(2)}"  # SN folders named SN_01, SN_02, ...

    # Evening recordings of the current date
    current_date_folder = os.path.join(base_dir, date_folders[i])
    for wav_file in os.listdir(current_date_folder):
        if '180000' <= wav_file.split('_')[1][:6] <= '235000':
            source = os.path.join(current_date_folder, wav_file)
            destination = os.path.join(base_dir, sn_folder, wav_file)
            shutil.move(source, destination)

    # Morning recordings of the next date
    next_date_folder = os.path.join(base_dir, date_folders[i+1])
    for wav_file in os.listdir(next_date_folder):
        if '000000' <= wav_file.split('_')[1][:6] <= '072000':
            source = os.path.join(next_date_folder, wav_file)
            destination = os.path.join(base_dir, sn_folder, wav_file)
            shutil.move(source, destination)

    if i == 17:  # Adjust this number based on the number of date folder you have
        break

print("Reorganization into survey occasions completed.")


Reorganization into survey occasions completed.


### Código para reordenar las grabaciones en SN automaticamente 

In [3]:
import os
import shutil

# Define the base directory containing all site folders
base_dir = '/mnt/d/night_recordings/'  # Update with your actual path to night_recordings

# Iterate through each site folder within the base directory
for site_folder in os.listdir(base_dir):
    site_path = os.path.join(base_dir, site_folder)

    # Check if it's a directory (to skip any files in the base directory)
    if not os.path.isdir(site_path):
        continue

    # Get a sorted list of all date folders within the site folder
    date_folders = sorted([d for d in os.listdir(site_path) if os.path.isdir(os.path.join(site_path, d)) and d.isdigit()])

    # Iterate over the date folders and pair them for each SN folder
    for i in range(len(date_folders)-1):
        sn_folder = f"SN_{str(i+1).zfill(2)}"  # SN folders named SN_01, SN_02, ...

        # Evening recordings of the current date
        current_date_folder = os.path.join(site_path, date_folders[i])
        for wav_file in os.listdir(current_date_folder):
            if '180000' <= wav_file.split('_')[1][:6] <= '235000':
                source = os.path.join(current_date_folder, wav_file)
                destination = os.path.join(site_path, sn_folder, wav_file)
                shutil.move(source, destination)

        # Morning recordings of the next date
        next_date_folder = os.path.join(site_path, date_folders[i+1])
        for wav_file in os.listdir(next_date_folder):
            if '000000' <= wav_file.split('_')[1][:6] <= '072000':
                source = os.path.join(next_date_folder, wav_file)
                destination = os.path.join(site_path, sn_folder, wav_file)
                shutil.move(source, destination)

    print(f"Reorganization into survey occasions completed for {site_folder}.")

print("All sites have been processed.")


Reorganization into survey occasions completed for P16-ARU25.
Reorganization into survey occasions completed for P17-ARU21.
Reorganization into survey occasions completed for P19-ARU9.
Reorganization into survey occasions completed for P20-ARU19.
Reorganization into survey occasions completed for P22-ARU11.
Reorganization into survey occasions completed for P23-ARU5.
Reorganization into survey occasions completed for P15-ARU8.
Reorganization into survey occasions completed for P5-ARU17.
Reorganization into survey occasions completed for P6-ARU10.
Reorganization into survey occasions completed for P8-ARU9.
Reorganization into survey occasions completed for P11-ARU28.
Reorganization into survey occasions completed for P12-ARU20.
Reorganization into survey occasions completed for P13-ARU29.
Reorganization into survey occasions completed for P14-ARU27.
Reorganization into survey occasions completed for P1-ARU11.
Reorganization into survey occasions completed for P2-ARU05.
Reorganization in

### Codigo para renumerar las SN folders

In [10]:
import os

def rename_sn_folders(base_dir, add_number):
    # Get a list of all SN folders in the base directory
    sn_folders = [f for f in os.listdir(base_dir) if f.startswith('SN_') and os.path.isdir(os.path.join(base_dir, f))]

    # Sort the folders to ensure they are processed in order
    sn_folders.sort(reverse=True)

    # Calculate new folder names and store them in a dictionary
    new_names = {}
    for folder in sn_folders:
        # Extract the current sequence number
        seq_num = int(folder.split('_')[1])
        # Calculate the new sequence number
        new_seq_num = seq_num + add_number
        # Store the new folder name in the dictionary
        new_names[folder] = 'SN_{:02d}'.format(new_seq_num)

    # Rename the folders
    for old_name, new_name in new_names.items():
        os.rename(os.path.join(base_dir, old_name), os.path.join(base_dir, new_name))

# Specify the base directory and the number to add
base_dir = '/mnt/d/test_night_recordings/M9-ARU4-1207-2907/'  # Replace with your actual base directory path
add_number = -1  # Replace with the number you want to add

# Call the function to rename the SN folders
rename_sn_folders(base_dir, add_number)


OSError: [Errno 39] Directory not empty: '/mnt/d/test_night_recordings/M9-ARU4-1207-2907/SN_19' -> '/mnt/d/test_night_recordings/M9-ARU4-1207-2907/SN_18'

### Otro codigo para renumerar las SN folders de 01 sucesivamente

In [7]:
import os

def renumber_sn_folders(base_dir):
    # Get a list of all SN folders in the base directory
    sn_folders = [f for f in os.listdir(base_dir) if f.startswith('SN_') and os.path.isdir(os.path.join(base_dir, f))]

    # Sort the folders to ensure they are processed in the original order
    sn_folders.sort()

    # Dictionary to store the new folder names
    new_names = {}

    # Generate new folder names starting from 'SN_01'
    for i, folder in enumerate(sn_folders, start=1):
        new_name = f'SN_{i:02d}'
        new_names[folder] = new_name

    # Rename the folders
    for old_name, new_name in new_names.items():
        os.rename(os.path.join(base_dir, old_name), os.path.join(base_dir, new_name))

# Specify the base directory
base_dir = '/mnt/d/night_recordings/P12-ARU20/'  # Replace with your actual base directory path

# Call the function to renumber the SN folders
renumber_sn_folders(base_dir)
